In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from researchlib.single_import import *

Using GPU 0: GeForce RTX 2080 Ti
Driver: 418.56


In [3]:
train_loader = VisionDataset(vision.FashionMNIST, 128, normalize=False)

In [4]:
g = builder([
    nn.Linear(110, 8*4*4),
    nn.BatchNorm1d(8*4*4),
    nn.LeakyReLU(),
    layer.Reshape((-1, 8, 4, 4)),
    AutoConvTransposeNet2d(8, 3, 'vgg', start_filter=128, min_filter=128, preact=False, pooling_type='interpolate', norm='batch'),
    nn.Conv2d(128, 1, 5),
    nn.Sigmoid(),
])  

8 128
128 128
128 128


In [ ]:
d = builder([
    layer.ConditionProjection(
        builder([
            nn.Linear(10, 784),
            nn.BatchNorm1d(784),
            nn.LeakyReLU(),
            layer.Reshape((-1, 1, 28, 28)),
        ])
    ),
    AutoConvNet2d(2, 4, 'vgg', start_filter=128, max_filter=128, flatten=True, pooling_type='maxpool', preact=False, norm='batch'),
#     Auxiliary(builder([
#         nn.Linear(128, 10),
#         nn.LogSoftmax(-1)
#     ])),
    nn.Linear(128, 1),
#     nn.Sigmoid()
])

2 128
128 128
128 128
128 128


In [ ]:
runner = Runner(GANModel(g, d, latent_vector_len=100, condition_vector_len=[10, 10], condition_onehot=True),
                train_loader, 
                None, 
                'rmsprop', 
                GANLoss('wgan'),
                monitor_state='g_loss',
                monitor_mode='min')

In [ ]:
runner.init_model('lsuv')

In [ ]:
runner.fit(4)

  Epochs   train_d_loss  train_g_loss 
    1*       -0.7985        0.6798    


    2        -1.2814        1.1644    


In [ ]:
img = runner.model.sample(100, condition_data=list(range(10))*10)
import matplotlib.pyplot as plt
fig, arr = plt.subplots(10, 10, figsize=(20, 20))
for i in range(100):
    r = img[i][0].detach().cpu().numpy()#.transpose(1,2,0)
#     r -= r.min()
#     r /= r.max()
    arr[i//10][i%10].axis('off')
    arr[i//10][i%10].imshow(r, cmap='gray')
plt.tight_layout()
plt.show()

In [ ]:
runner.report()